In [56]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 250 kB 7.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [64]:
import pandas as pd

# Load data
nifty = pd.read_csv("Niftweekly.csv")
sensex = pd.read_csv("SensexWeekly.csv")

for df in [nifty, sensex]:
    df.rename(columns={'Change %': 'Change'}, inplace=True)
    df.drop(columns=['Open', 'High', 'Low'], inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
    df['Change'] = df['Change'].str.replace('%', '', regex=False).astype(float)
    df['Price'] = df['Price'].str.replace(',', '', regex=False).astype(float)
    df['Year'] = df['Date'].dt.year

#reverse the table bottom to top for nifty and sensex according to date
nifty = nifty.sort_values('Date', ascending=True).reset_index(drop=True)
sensex = sensex.sort_values('Date', ascending=True).reset_index(drop=True)

print(nifty.head())
print(sensex.head())
print(nifty.shape)
print(sensex.shape)


        Date    Price     Vol.  Change  Year
0 2000-01-02  1613.30  231.89M    8.97  2000
1 2000-01-09  1622.75  220.53M    0.59  2000
2 2000-01-16  1620.60  205.57M   -0.13  2000
3 2000-01-23  1599.10  156.28M   -1.33  2000
4 2000-01-30  1599.75  183.96M    0.04  2000
        Date   Price Vol.  Change  Year
0 1990-01-07  780.34  NaN    0.74  1990
1 1990-01-14  710.75  NaN   -8.92  1990
2 1990-01-21  716.23  NaN    0.77  1990
3 1990-01-28  684.48  NaN   -4.43  1990
4 1990-02-04  659.30  NaN   -3.68  1990
(1305, 5)
(1819, 5)


In [65]:
# Calculate the number of weeks with a fall greater than 3% for Nifty and Sensex
# Filter weeks where absolute change is less than 3%
nifty_less3 = nifty[nifty['Change'] < -3]
sensex_less3 = sensex[sensex['Change'] < -3]
print(nifty_less3.shape)
print(sensex_less3.shape)

# Group and count
nifty_count = nifty_less3.groupby('Year').size().reset_index(name='Nifty <3% Weeks')
sensex_count = sensex_less3.groupby('Year').size().reset_index(name='Sensex <3% Weeks')

#reverse the table bottom to top for nifty and sensex according to date

(132, 5)
(231, 5)


In [66]:

# Function to simulate strategies per year
def simulate_sip_strategies(df, index_name, fall_threshold=-3):
    years = df['Year'].unique()
    current_price = df.iloc[-1]['Price']  # Use latest price in dataset
    sip_logs = []

    for year in years:
        # SIP strategy
        sip_units, sip_invested = 0, 0
        for month in range(1, 13):
            month_data = df[(df['Year'] == year) & (df['Date'].dt.month == month)]
            if not month_data.empty:
                row = month_data.iloc[0]
                units = 10000 / row['Price']
                sip_units += units
                sip_invested += 10000
                sip_logs.append({
                    'Strategy': 'SIP',
                    'Date': row['Date'],
                    'Price': row['Price'],
                    'Amount Invested': 10000,
                    'Units': units,
                    'Current Value': units * current_price,
                    'Running Total Investment': sip_invested,
                    'Running Corpus Value': sip_units * current_price,
                    'Index': index_name,
                    'Year': year
                })

    return pd.DataFrame(sip_logs)
def simulate_dip_strategies(df, index_name, fall_threshold=-3):
    years = df['Year'].unique()
    print (years)
    current_price = df.iloc[-1]['Price']  # Use latest price in dataset
    dip_logs = []
    for year in years:
        # DIP strategy
        dip_data = df[(df['Year'] == year) & (df['Change'] < fall_threshold)]
        dip_units, dip_invested = 0, 0
        dip_count = 0

        for _, row in dip_data.iterrows():
            if dip_count >= 12:
                break
            units = 10000 / row['Price']
            dip_units += units
            dip_invested += 10000
            dip_logs.append({
                'Strategy': 'DIP',
                'Date': row['Date'],
                'Price': row['Price'],
                'Amount Invested': 10000,
                'Units': units,
                'Current Value': units * current_price,
                'Running Total Investment': dip_invested,
                'Running Corpus Value': dip_units * current_price,
                'Index': index_name,
                'Year': year
            })
            dip_count += 1

        if dip_count < 12:
            dec_data = df[(df['Year'] == year) & (df['Date'].dt.month == 12)]
            if not dec_data.empty:
                dec_row = dec_data.iloc[-1]
                for _ in range(12 - dip_count):
                    units = 10000 / dec_row['Price']
                    dip_units += units
                    dip_invested += 10000
                    dip_logs.append({
                        'Strategy': 'DEC_FILL',
                        'Date': dec_row['Date'],
                        'Price': dec_row['Price'],
                        'Amount Invested': 10000,
                        'Units': units,
                        'Current Value': units * current_price,
                        'Running Total Investment': dip_invested,
                        'Running Corpus Value': dip_units * current_price,
                        'Index': index_name,
                        'Year': year
                    })
    return pd.DataFrame(dip_logs)

# Generate full logs for both indices
fall_threshold = -5  # Configurable
nifty_log_sip = simulate_sip_strategies(nifty, "Nifty", fall_threshold)
nifty_log_dip = simulate_dip_strategies(nifty, "Nifty", fall_threshold)
sensex_log_sip = simulate_sip_strategies(sensex, "Sensex", fall_threshold)
sensex_log_dip = simulate_dip_strategies(sensex, "Sensex", fall_threshold)
with pd.ExcelWriter("dip_vs_sip_analysis.xlsx", engine="openpyxl") as writer:
    nifty_log_sip.to_excel(writer, sheet_name="Nifty_SIP", index=False)
    nifty_log_dip.to_excel(writer, sheet_name="Nifty_Dip", index=False)
    sensex_log_sip.to_excel(writer, sheet_name="Sensex_SIP", index=False)
    sensex_log_dip.to_excel(writer, sheet_name="Sensex_Dip", index=False)



[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020 2021 2022 2023 2024]
